In [26]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from skopt import gp_minimize
from skopt.space import Integer
from numpy import mean
from skopt.utils import use_named_args
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, explained_variance_score, median_absolute_error
#
from sklearn.neighbors import KNeighborsRegressor
import numpy as np
#

In [ ]:
#dataset = pd.read_csv('D:/Code/Python/NCKH/A1/c.csv')
dataset = pd.read_csv('./Dataset/para.csv')
# 
features = ["Freq"]
targets = ["W", "L", "Lf"]
x = dataset[features]
y = dataset[targets]

In [28]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=1)

# Scaling
scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train)
x_test_scaled = scaler.transform(x_test)

In [29]:
search_space = [Integer(1, 100, name='n_neighbors'), Integer(1, 5, name='p')]

In [ ]:
@use_named_args(search_space)
def evaluate_model(**params):
    model = KNeighborsRegressor(n_neighbors=params['n_neighbors'], p=params['p'])
    model.fit(x_train_scaled, y_train)
    y_pred = model.predict(x_test_scaled)
    error = mean((y_test - y_pred) ** 2)  # MSE
    return error

In [31]:
result = gp_minimize(evaluate_model, search_space)

C:\Users\sachxn\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\LocalCache\local-packages\Python38\site-packages\skopt\optimizer\optimizer.py:517: UserWarning: The objective has been evaluated at point [1, 5] before, using random point [21, 1]
  warnings.warn(
C:\Users\sachxn\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\LocalCache\local-packages\Python38\site-packages\skopt\optimizer\optimizer.py:517: UserWarning: The objective has been evaluated at point [1, 5] before, using random point [16, 3]
  warnings.warn(
C:\Users\sachxn\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\LocalCache\local-packages\Python38\site-packages\skopt\optimizer\optimizer.py:517: UserWarning: The objective has been evaluated at point [1, 5] before, using random point [67, 5]
  warnings.warn(
C:\Users\sachxn\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\LocalCache\local-packages\Python38\site-packages\skop

In [32]:
best_n_neighbors = result.x[0]
best_p = result.x[1]
best_model = KNeighborsRegressor(n_neighbors=best_n_neighbors, p=best_p)
best_model.fit(x_train_scaled, y_train)

KNeighborsRegressor(n_neighbors=2)

In [ ]:
y_pred = best_model.predict(new_data_scaled)  # Thêm dòng này để định nghĩa biến y_pred
# 
predictions = y_pred  # Giả sử y_pred là kết quả dự đoán của bạn

# DataFrame 
predictions_df = pd.DataFrame(predictions, columns=['Predictions'])

# 
predictions_df.to_csv('predictions.csv', index=False)


'""\ny_pred = best_model.predict(new_data_scaled)  # Thêm dòng này để định nghĩa biến y_pred\n# Dữ liệu dự đoán\npredictions = y_pred  # Giả sử y_pred là kết quả dự đoán của bạn\n\n# Tạo DataFrame từ dữ liệu dự đoán\npredictions_df = pd.DataFrame(predictions, columns=[\'Predictions\'])\n\n# Ghi dữ liệu dự đoán ra một tệp tin CSV mới\npredictions_df.to_csv(\'predictions.csv\', index=False)\n'

In [ ]:
"""""
new_dataset = pd.read_csv('D:/Code/Python/NCKH/A1/c.csv')  # 
new_data = new_dataset[["Freq","W", "L", "WL", "Lf"]]
new_scaler = StandardScaler()
new_data_scaled = new_scaler.fit_transform(new_data)
y_pred = best_model.predict(new_data_scaled)
"""""
new_dataset = pd.read_csv('D:/Code/Python/NCKH/A1/c.csv')  # 
new_data = new_dataset["Freq"].values.reshape(-1, 1)  # C

# Scaling 
new_data_scaled = scaler.transform(new_data)

# 
y_pred = best_model.predict(x_test_scaled)


C:\Users\sachxn\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\LocalCache\local-packages\Python38\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [ ]:
print('Best Accuracy: %.3f' % (1.0 - result.fun))
print('Best Parameters: n_neighbors=%d, p=%d' % (result.x[0], result.x[1]))

# 

print("Predictions:", y_pred)

Best Accuracy: 0.997
Best Parameters: n_neighbors=2, p=2
Predictions: [[ 25.575  19.585  11.52 ]
 [ 49.205  38.245  22.165]
 [110.15   86.135  49.635]
 [ 38.41   29.74   17.305]
 [ 25.445  19.485  11.46 ]
 [ 17.165  12.89    7.73 ]
 [ 26.835  20.59   12.085]
 [ 44.805  34.785  20.185]
 [ 29.4    22.615  13.245]
 [ 43.275  33.575  19.495]
 [ 38.755  30.005  17.455]
 [100.45   78.515  45.26 ]
 [ 35.485  27.425  15.985]
 [ 39.22   30.375  17.665]
 [ 24.71   18.9    11.13 ]
 [ 95.32   74.48   42.945]
 [ 27.105  20.795  12.21 ]
 [ 37.805  29.26   17.03 ]
 [ 28.005  21.51   12.615]
 [ 25.51   19.53   11.49 ]
 [ 49.795  38.71   22.435]
 [ 35.33   27.305  15.915]
 [ 17.345  13.035   7.81 ]
 [ 17.14   12.87    7.72 ]
 [ 51.42   39.995  23.165]
 [ 16.615  12.45    7.485]
 [ 34.025  26.28   15.325]
 [107.35   83.895  48.35 ]
 [103.2    80.655  46.485]
 [ 18.03   13.58    8.12 ]
 [ 34.375  26.55   15.485]
 [ 38.475  29.79   17.335]
 [ 93.09   72.725  41.935]
 [ 44.715  34.715  20.145]
 [ 51.135  3

In [36]:
# """""
# FreqInput = float(input('nhập vào tấn số muốn dự đoán:'))
# datapre = regressor.predict([[FreqInput]])
# regressor.predict([[FreqInput]])
# """""
# # Nhập giá trị tần số muốn dự đoán từ người dùng
# FreqInput = float(input('Nhập vào tần số muốn dự đoán: '))

# # Chuyển giá trị tần số đã nhập thành một mảng 2D có dạng [[FreqInput]]
# new_data = [[FreqInput]]

# # Scaling dữ liệu mới
# new_data_scaled = scaler.transform(new_data)

# # Dự đoán giá trị y tương ứng với giá trị tần số đã nhập
# y_pred = best_model.predict(new_data_scaled)

# # In ra kết quả dự đoán
# print('Giá trị nhập vào là: ', FreqInput)
# print('Giá trị dự đoán:', y_pred)

In [ ]:
def regression_report(y_true, y_pred):
    mae = mean_absolute_error(y_true, y_pred)
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_true, y_pred)
    explained_variance = explained_variance_score(y_true, y_pred)
    median_ae = median_absolute_error(y_true, y_pred)

    # Mean Bias Deviation (MBD)
    mbd = np.mean(y_pred - y_true)

    # Mean Absolute Percentage Error (MAPE)
    mape = np.mean(np.abs((y_true - y_pred) / y_true)) * 100

    # Symmetric Mean Absolute Percentage Error (sMAPE)
    smape = np.mean(2 * np.abs(y_pred - y_true) / (np.abs(y_true) + np.abs(y_pred))) * 100

    report = pd.DataFrame({
        'Metric': [
            'Mean Absolute Error (MAE)',
            'Mean Squared Error (MSE)',
            'Root Mean Squared Error (RMSE)',
            'R² Score',
            'Explained Variance Score',
            'Median Absolute Error',
            'Mean Bias Deviation (MBD)',
            'Mean Absolute Percentage Error (MAPE)',
            'Symmetric Mean Absolute Percentage Error (sMAPE)'
        ],
        'Gausians': [mae, mse, rmse, r2, explained_variance, median_ae, mbd, mape, smape]
    })

    return report

report = regression_report(y_test, y_pred)
print(report)

                                             Metric  Gausians
0                         Mean Absolute Error (MAE)  0.017706
1                          Mean Squared Error (MSE)  0.002587
2                    Root Mean Squared Error (RMSE)  0.050860
3                                          R² Score  0.999994
4                          Explained Variance Score  0.999994
5                             Median Absolute Error  0.005000
6                         Mean Bias Deviation (MBD) -0.002981
7             Mean Absolute Percentage Error (MAPE)  0.044457
8  Symmetric Mean Absolute Percentage Error (sMAPE)  0.044462
